Tout d'abord on crée la fonction

In [152]:
import numpy as np
import random
import torch
torch.set_default_dtype(torch.float64)
from scipy.sparse.linalg import LinearOperator
from scipy.special import softmax

class Problem:
    def __init__(self, U, V, mu, nu, q, epsilon):
        self.U = U  # (nb_receivers, nb_states, nb_actions)
        self.V = V  # (nb_states, nb_actions)
        self.mu = mu  # (nb_states,)
        self.nu = nu  # (nb_receivers, nb_states)
        self.q = q  # (nb_receivers, nb_messages, nb_actions)
        self.epsilon = epsilon  # (nb_receivers,)
        self.nb_receivers, self.nb_states, self.nb_actions = U.shape
        self.nb_messages = q.shape[1]
        self.size = self.nb_receivers * self.nb_states * self.nb_messages
        self.shape = (self.nb_receivers, self.nb_states, self.nb_messages)
        self.check()
    
    def check(self):
        for receiver_idx in range(self.nb_receivers):
            self.debug_shape(self.U[receiver_idx], [self.nb_states, self.nb_actions])
            self.debug_shape(self.V, [self.nb_states, self.nb_actions])
            self.debug_shape(self.mu, [self.nb_states])
            self.debug_shape(self.nu[receiver_idx], [self.nb_states])
            self.debug_shape(self.q[receiver_idx], [self.nb_messages, self.nb_actions])

    def debug_shape(self, vect, target_shape):
        if list(vect.shape) != target_shape:
            print(f"Found vector of size {vect.shape}, expected {target_shape}")
            assert False
    
    def verbose(self, pi):
        print(f"We have (state, message, action) = ({self.nb_states}, {self.nb_messages}, {self.nb_actions})")
        self.debug_shape(pi, [self.nb_states, self.nb_messages])
        for receiver_idx in range(self.nb_receivers):
            theta = self.compute_theta(self.compute_g(pi, receiver_idx), receiver_idx)
            self.debug_shape(theta, [self.nb_messages, self.nb_actions])
            print(f"We have 1 = {pi.sum(axis=1)}")
            print(f"We have 1 = {theta.sum(axis=1)}")
            print("Theta", theta)
        print("Objective", self.objective(pi, range(self.nb_receivers)))
    
    def compute_g(self, pi, receiver_idx):
        denominator = (pi * self.nu[receiver_idx][:, None]).sum(axis=0)
        self.debug_shape(denominator, [self.nb_messages])
        g = (pi[:, :, None] * self.nu[receiver_idx][:, None, None] * self.U[receiver_idx][:, None, :]).sum(axis=0)
        self.debug_shape(g, [self.nb_messages, self.nb_actions])
        return g / denominator[:, None]

    def compute_theta(self, g, receiver_idx):
        max_g, _ = g.max(axis=1)
        exp = torch.exp((g - max_g[:, None]) / self.epsilon[receiver_idx])
        self.debug_shape(exp, [self.nb_messages, self.nb_actions])
        theta = self.q[receiver_idx] * exp
        denom = theta.sum(axis=1)
        return theta / denom[:, None]

    def objective(self, pi, receivers_batch):
        total_objective = 0
        for receiver_idx in receivers_batch:
            g = self.compute_g(pi, receiver_idx)
            theta = self.compute_theta(g, receiver_idx)
            total_objective += (theta[None, :, :] * pi[:, :, None] * self.mu[:, None, None] * self.V[:, None, :]).sum()
        return total_objective / len(receivers_batch)

    def value(self, x, receivers_batch=None):
        if receivers_batch is None:
            receivers_batch = range(self.nb_receivers)
        x = x.reshape(self.nb_states, self.nb_messages)
        x = torch.from_numpy(x).requires_grad_(True)
        f = self.objective(x, receivers_batch)
        f.backward()
        df = x.grad
        return -f.item(), -df.numpy()

    def project(self, x):
        x = torch.from_numpy(x).reshape(self.nb_states, self.nb_messages)
        x_projected = torch.zeros_like(x)
        for i in range(x.shape[0]):
            row = x[i, :]
            sorted_row, _ = torch.sort(row, descending=True)
            cumulative_sum = torch.cumsum(sorted_row, dim=0)
            rho = torch.nonzero(sorted_row * torch.arange(1, len(row) + 1) > (cumulative_sum - 1), as_tuple=False).max()
            theta = (cumulative_sum[rho] - 1) / (rho + 1)
            x_projected[i, :] = torch.clamp(row - theta, min=0)
        return x_projected.numpy()

    def project_tangent(self, x, d):
        d2 = d - d.mean(axis=1)[:, None]
        d2[(x == 0) * (d2 < 0)] = 0.
        d2[(x == 1) * (d2 > 0)] = 0.
        return d2


In [166]:


def dot(a, b):
    return (a * b).sum()

def ls_wolfe(x, function, step, descent, f, df, batch):
    """
    Wolfe line search for stochastic gradient descent.
    """
    step_min, step_max = 0., np.inf
    scal = dot(df, descent)
    if scal > 0:
        print('WARNING with scal', scal)
    step2 = step
    eps1, eps2 = 1.e-4, 0.9  # Wolfe condition parameters
    i = 0
    while i < 100:
        i += 1
        x2 = function.project(x + step2 * descent)
        f2, df2 = function.value(x2, batch)
        if dot(x2 - x, df) >= 0:
            print('We have a problem', dot(x2 - x, df), dot(descent, df))
        if f2 > f + eps1 * dot(x2 - x, df):  # step is too big, decrease it
            step_max = step2
            step2 = 0.5 * (step_min + step_max)
        else:
            if dot(df2, x2 - x) < eps2 * dot(df, x2 - x):  # step is too small, increase it
                step_min = step2
                step2 = min(0.5 * (step_min + step_max), 2 * step_min)
            else:
                return x2, f2, df2, step2
    print('We do not exit Wolfe')
    return x2, f2, df2, step2


def dot(a,b) :
    return (a*b).sum()

def ls_wolfe(x,function,step,descent,f,df,batch) :
    step_min,step_max=0.,np.inf
    scal=dot(df,descent)
    if scal > 0 :
        print('WARNING with scal',scal)
    step2=step
    eps1,eps2=1.e-4,0.9
    i=0
    while i<100 :
        i=i+1
        x2=function.project(x+step2*descent)
        f2,df2=function.value(x2,batch)
        if dot(x2-x,df) >=0 :
            print('We have a problem',dot(x2-x,df),dot(descent,df))
        if f2>f+eps1*dot(x2-x,df) : # step is too big, decrease it
            step_max=step2
            step2=0.5*(step_min+step_max)
        else :
            if dot(df2,x2-x) < eps2*dot(df,x2-x) : # step is too small, increase it
                step_min=step2
                step2=min(0.5*(step_min+step_max),2*step_min)
            else :
                return x2,f2,df2,step2
    print('We do not exit Wolfe')
    print(f2>f+eps1*step2*scal,dot(df2,descent) < eps2*scal)
    return x2,f2,df2,step2




def optimize(function,itermax = 5000,tol=1.e-6,batch_size = 100,verbose=True):
    np.random.seed(42)
    receivers = list(range(function.nb_receivers))
    x = np.random.randn(function.nb_states, function.nb_messages)
    x=function.project(x)
    np.random.seed(None)
    list_costs=[]
    list_grads=[]
    nbiter = 0
    batch = np.random.choice(receivers, size=batch_size, replace=False)
    f,df=function.value(x,batch)
    df_tangent=function.project_tangent(x,-df)
    norm_grad=np.linalg.norm(df_tangent)
    err=2*tol
    if verbose :
        print('iter={:4d} f={:1.3e} df={:1.3e}'.format(nbiter,f,err))
    list_costs.append(f)
    list_grads.append(norm_grad)
    while (err > tol) and (nbiter < itermax):
        descent=-df
        x_old=np.copy(x)
        x,f,df,step = ls_wolfe(x, function,1., descent,f,df,batch)
        batch = np.random.choice(receivers, size=batch_size, replace=False)
        norm_grad = np.linalg.norm(function.project_tangent(x,-df))
        list_costs.append(f)
        list_grads.append(norm_grad)
        err=norm_grad
        nbiter+=1
        if verbose :
            print('iter={:4d} f={:1.3e} err={:1.3e} s={:1.3e}'.format(nbiter,f,err,step))
        if (err <= tol):
            if verbose : print("Success !!! Algorithm converged !!!")
            return x,list_costs,list_grads
    if verbose : print("FAILED to converge")
        

def stochastic_optimize(function,itermax = 5000,tol=1.e-6,batch_size = 100,verbose=True):
    np.random.seed(42)
    receivers = list(range(function.nb_receivers))
    x = np.random.randn(function.nb_states, function.nb_messages)
    x=function.project(x)
    np.random.seed(None)
    list_costs=[]
    list_grads=[]
    nbiter = 0
    batch = np.random.choice(receivers, size=batch_size, replace=False)
    f,df=function.value(x,batch)
    df_tangent=function.project_tangent(x,-df)
    norm_grad=np.linalg.norm(df_tangent)
    err=2*tol
    if verbose :
        print('iter={:4d} f={:1.3e} df={:1.3e}'.format(nbiter,f,err))
    list_costs.append(f)
    list_grads.append(norm_grad)
    while (err > tol) and (nbiter < itermax):
        nbiter+=1
        x_old=np.copy(x)
        batch = np.random.choice(receivers, size=batch_size, replace=False)
        x = function.project(x - (1/nbiter)*df)
        f,df = function.value(x,batch)
        norm_grad = np.linalg.norm(function.project_tangent(x,-df))
        list_costs.append(f)
        list_grads.append(norm_grad)
        err=norm_grad
        if verbose :
            print('iter={:4d} f={:1.3e} err={:1.3e}'.format(nbiter,f,err))
        if (err <= tol):
            if verbose : print("Success !!! Algorithm converged !!!")
            return x,list_costs,list_grads
    if verbose : print("FAILED to converge")



On crée 1000 variations d'un receiver de base

In [167]:
alpha = 0.7
beta = 0.3

# Initialisation des matrices de base
nb_receivers = 1000
nb_states = 2
nb_actions = 2
nb_messages = 2

U_base = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.float64)  # Matrice U de base
nu_base = torch.tensor([alpha, 1 - alpha], dtype=torch.float64)  # Distribution de probabilité de base pour nu
q_base = torch.tensor([beta, 1 - beta], dtype=torch.float64)  # Une seule ligne pour q de base
epsilon_base = 0.01  # Valeur de base pour epsilon

mu = torch.tensor([alpha, 1 - alpha])

# Fonction pour ajouter du bruit à une matrice tout en conservant les propriétés des distributions
def add_noise_to_distribution(base_vector, noise_level):
    noisy_vector = base_vector + noise_level * torch.randn_like(base_vector)
    noisy_vector = torch.clamp(noisy_vector, min=1e-8)  # Évite les valeurs négatives
    return noisy_vector / noisy_vector.sum()  # Normalisation pour conserver les distributions

# Génération des variations
noise_level_U = 0.01
noise_level_nu = 0.01
noise_level_q = 0.01
noise_level_epsilon = 0.01

U = torch.stack([U_base + noise_level_U * torch.randn_like(U_base) for _ in range(nb_receivers)])
nu = torch.stack([add_noise_to_distribution(nu_base, noise_level_nu) for _ in range(nb_receivers)])
q = torch.stack([add_noise_to_distribution(q_base, noise_level_q).expand(nb_messages, -1) for _ in range(nb_receivers)])
epsilon = torch.tensor([epsilon_base + noise_level_epsilon * np.random.randn() for _ in range(nb_receivers)])
epsilon = torch.clamp(epsilon, min=1e-8)  # Assurez-vous que epsilon reste positif

# Vérifications
print("Exemple de U :", U[0])
print("Exemple de nu (somme = 1) :", nu[0], "Somme =", nu[0].sum())
print("Exemple de q (lignes identiques, somme = 1) :", q[0], "Somme des lignes =", q[0].sum(dim=1))
print("Exemple de epsilon :", epsilon[:10])

V = torch.tensor([[0.0, 1.0],
                  [0.0, 1.0]])



Exemple de U : tensor([[1.0049, 0.0118],
        [0.0071, 0.9800]])
Exemple de nu (somme = 1) : tensor([0.6883, 0.3117]) Somme = tensor(1.)
Exemple de q (lignes identiques, somme = 1) : tensor([[0.3039, 0.6961],
        [0.3039, 0.6961]]) Somme des lignes = tensor([1., 1.])
Exemple de epsilon : tensor([1.8902e-02, 8.7466e-03, 1.0000e-08, 9.4291e-03, 1.4657e-02, 4.0578e-04,
        1.8641e-02, 1.9156e-02, 2.7247e-02, 6.3527e-03])


On optimise la "vraie" fonction (moyenne sur tous les Receivers) pour comparer ensuite avec l'optimisation sur des batchs

In [168]:
# Création de l'objet Problem avec les données générées
P = Problem(U, V, mu, nu, q, epsilon)

x,costs,grad=stochastic_optimize(P,tol=1.e-5,verbose=True,batch_size = 1000)

iter=   0 f=-4.091e-01 df=2.000e-05
iter=   1 f=-6.257e-03 err=1.583e-01
iter=   2 f=-5.535e-02 err=1.582e+00
iter=   3 f=-3.726e-01 err=4.950e-01
iter=   4 f=-4.338e-01 err=4.950e-01
iter=   5 f=-4.828e-01 err=4.947e-01
iter=   6 f=-5.235e-01 err=4.877e-01
iter=   7 f=-5.545e-01 err=3.254e-01
iter=   8 f=-5.451e-01 err=1.217e+00
iter=   9 f=-5.112e-01 err=4.924e-01
iter=  10 f=-5.352e-01 err=4.745e-01
iter=  11 f=-5.538e-01 err=3.437e-01
iter=  12 f=-5.573e-01 err=2.152e-01
iter=  13 f=-5.572e-01 err=2.049e-01
iter=  14 f=-5.577e-01 err=1.578e-01
iter=  15 f=-5.579e-01 err=1.226e-01
iter=  16 f=-5.580e-01 err=8.612e-02
iter=  17 f=-5.581e-01 err=5.457e-02
iter=  18 f=-5.582e-01 err=3.277e-02
iter=  19 f=-5.582e-01 err=1.679e-02
iter=  20 f=-5.582e-01 err=7.637e-03
iter=  21 f=-5.582e-01 err=2.908e-03
iter=  22 f=-5.582e-01 err=9.342e-04
iter=  23 f=-5.582e-01 err=2.455e-04
iter=  24 f=-5.582e-01 err=5.167e-05
iter=  25 f=-5.582e-01 err=8.371e-06
Success !!! Algorithm converged !!!


In [169]:
x

array([[0.61818672, 0.38181328],
       [0.        , 1.        ]])

In [170]:
P.value(x)

(-0.5581882865231921,
 array([[-1.52216288e-12, -1.18386294e-05],
        [-2.81973319e-11, -5.58183766e-01]]))

In [171]:
y,costs,grad=stochastic_optimize(P,tol=1.e-3,verbose=True,batch_size = 100)

iter=   0 f=-4.091e-01 df=2.000e-03
iter=   1 f=-6.013e-03 err=1.726e-01
iter=   2 f=-7.460e-02 err=2.111e+00
iter=   3 f=-3.000e-01 err=4.950e-01
iter=   4 f=-3.612e-01 err=4.950e-01
iter=   5 f=-4.102e-01 err=4.950e-01
iter=   6 f=-4.511e-01 err=4.950e-01
iter=   7 f=-4.861e-01 err=4.947e-01
iter=   8 f=-5.166e-01 err=4.917e-01
iter=   9 f=-5.430e-01 err=4.625e-01
iter=  10 f=-5.585e-01 err=1.310e-01
iter=  11 f=-5.538e-01 err=6.029e-01
iter=  12 f=-5.463e-01 err=4.377e-01
iter=  13 f=-5.585e-01 err=2.590e-01
iter=  14 f=-5.542e-01 err=7.091e-01
iter=  15 f=-5.481e-01 err=4.168e-01
iter=  16 f=-5.576e-01 err=2.300e-01
iter=  17 f=-5.608e-01 err=4.543e-03
iter=  18 f=-5.584e-01 err=8.279e-02
iter=  19 f=-5.549e-01 err=2.291e-01
iter=  20 f=-5.578e-01 err=3.083e-01
iter=  21 f=-5.597e-01 err=2.483e-02
iter=  22 f=-5.563e-01 err=1.621e-01
iter=  23 f=-5.602e-01 err=2.008e-01
iter=  24 f=-5.569e-01 err=1.273e-01
iter=  25 f=-5.576e-01 err=4.779e-02
iter=  26 f=-5.566e-01 err=5.370e-02
it

In [172]:
y,x

(array([[0.61795114, 0.38204886],
        [0.        , 1.        ]]),
 array([[0.61818672, 0.38181328],
        [0.        , 1.        ]]))

In [173]:
P.value(y),P.value(x)

((-0.5581866810809846,
  array([[-1.52216288e-12,  1.36225261e-02],
         [-2.81973319e-11, -5.63391152e-01]])),
 (-0.5581882865231921,
  array([[-1.52216288e-12, -1.18386294e-05],
         [-2.81973319e-11, -5.58183766e-01]])))